In [1]:
import torch
import torch.nn as nn
import torch.optim as optimiser
from torch.utils.data import Dataset, DataLoader
import numpy as np 

In [14]:
X = torch.tensor([[0,0],[0,1],[1,0],[1,1]], dtype=torch.float32)
Y = torch.tensor([[0],[1],[1],[0]], dtype=torch.float32)

# Create Data Loader

class XDataset(Dataset):
    def __init__(self,X,Y):
        self.x = X
        self.y = Y
    def  __len__(self):
        return len(self.x)
    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]

class XOR(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2,3)
        self.layer2 = nn.Linear(3,1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self,x):
        x = self.sigmoid(self.layer1(x))
        x = self.sigmoid(self.layer2(x))
        return x
    
model = XOR()
criterion = nn.BCELoss()
opt = optimiser.Adam(model.parameters(),lr=0.001)

dataset = XDataset(X,Y)
dataloader = DataLoader(dataset,batch_size=2,shuffle = True)

epochs = 10000

for epoch in range(epochs):
    for inputs, targets in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs,targets)
        loss.backward()
        opt.step()
    if epoch%2000==0:
        print(f"Epoch: {epoch} : LOSS : {loss}")

with torch.no_grad():
    pred= model(dataset.x)
    pred = (pred > 0.5).float()
    print("Predicted values : ",pred)
    print("True Lables: ",dataset.y)

    

Epoch: 0 : LOSS : 0.6817018389701843
Epoch: 2000 : LOSS : 0.01992737129330635
Epoch: 4000 : LOSS : 0.02644157037138939
Epoch: 6000 : LOSS : 0.001718323677778244
Epoch: 8000 : LOSS : 0.001032185391522944
Predicted values :  tensor([[0.],
        [1.],
        [1.],
        [0.]])
True Lables:  tensor([[0.],
        [1.],
        [1.],
        [0.]])


In [19]:
class XOR2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2,3)
        self.layer2 = nn.Linear(3,1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self,x):
        x = self.relu(self.layer1(x))
        x = self.sigmoid(self.layer2(x))
        return x
    
model = XOR2()
criterion = nn.BCELoss()
opt = optimiser.Adam(model.parameters(),lr=0.001)

dataset = XDataset(X,Y)
dataloader = DataLoader(dataset,batch_size=2,shuffle = True)

epochs = 10000

for epoch in range(epochs):
    for inputs, targets in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs,targets)
        loss.backward()
        opt.step()
    if epoch%2000==0:
        print(f"Epoch: {epoch} : LOSS : {loss}")

with torch.no_grad():
    pred= model(dataset.x)
    pred = (pred > 0.5).float()
    print("Predicted values : ",pred)
    print("True Lables: ",dataset.y)


Epoch: 0 : LOSS : 0.6052368879318237
Epoch: 2000 : LOSS : 0.004101585131138563
Epoch: 4000 : LOSS : 5.9094978496432304e-05
Epoch: 6000 : LOSS : 4.804200415264859e-08
Epoch: 8000 : LOSS : 2.453912850697293e-09
Predicted values :  tensor([[0.],
        [1.],
        [1.],
        [0.]])
True Lables:  tensor([[0.],
        [1.],
        [1.],
        [0.]])


# Q4

In [19]:
import torch
import torch.nn as nn
import torch.optim as optimiser
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt


transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5),(0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class MnistModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28,168)
        self.fc2 = nn.Linear(168,64)
        self.final = nn.Linear(64,10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    def forward(self,x):
        x = x.view(-1,28*28)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.final(x)
        return x
    
model = MnistModel()
criterion = nn.CrossEntropyLoss()
optimizer = optimiser.Adam(model.parameters(),lr=0.0008)

epochs = 15

for epoch in range(epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch} LOSS: {loss}")

all_preds = []
all_labels = []
total = 0
correct = 0

with torch.no_grad():
    for images, labels in test_dataset:
        outputs = model(images)
        _,pred = torch.max(outputs,1)
        total+=1
        if pred==labels: correct+=1
    acc = correct/total*100
    print(f"Test accuracy : ",acc)





Epoch 0 LOSS: 0.5605297088623047
Epoch 1 LOSS: 0.2411869466304779
Epoch 2 LOSS: 0.1647346317768097
Epoch 3 LOSS: 0.012911248952150345
Epoch 4 LOSS: 0.06691393256187439
Epoch 5 LOSS: 0.11034650355577469
Epoch 6 LOSS: 0.05026650428771973
Epoch 7 LOSS: 0.06903780996799469
Epoch 8 LOSS: 0.03585657849907875
Epoch 9 LOSS: 0.180641308426857
Epoch 10 LOSS: 0.0029493868350982666
Epoch 11 LOSS: 0.021547872573137283
Epoch 12 LOSS: 0.017534825950860977
Epoch 13 LOSS: 0.017241377383470535
Epoch 14 LOSS: 0.06745678931474686
Test accuracy :  97.53


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt


# 1. Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 2. Create a Feed Forward Neural Network model with two hidden layers
class FFN_MNIST(nn.Module):
    def __init__(self):
        super(FFN_MNIST, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # Input layer to hidden layer 1 (784 to 128)
        self.fc2 = nn.Linear(128, 64)       # Hidden layer 1 to hidden layer 2 (128 to 64)
        self.fc3 = nn.Linear(64, 10)        # Hidden layer 2 to output layer (64 to 10 classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)    # Softmax for classification output
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the input image (batch_size, 28, 28) -> (batch_size, 784)
        x = self.relu(self.fc1(x))  # First hidden layer
        x = self.relu(self.fc2(x))  # Second hidden layer
        x = self.fc3(x)             # Output layer
        return x  # We will apply softmax when calculating probabilities (for confusion matrix)

# 3. Initialize the model, loss function, and optimizer
model = FFN_MNIST()
criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss combines Softmax and Negative Log Likelihood
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Train the model
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# 5. Test the model and compute accuracy and confusion matrix
model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_preds.extend(predicted.numpy())
        all_labels.extend(labels.numpy())

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")




Epoch [1/5], Loss: 0.3799
Epoch [2/5], Loss: 0.1799


KeyboardInterrupt: 